In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

x0 = 0.0    # initial x-position
y0 = 0.0    # initial y-position
dt = 0.10   # discrete time
Dr = 0.05   # rotational diffusion
v0 = 1.0    # velocity

numParticle = 5
numStep = 1000
sigma = np.sqrt(2.0*Dr*dt) # standard deviation for random angle change

fig, ax = plt.subplots()

xs = []
ys = []
thetas = []

# run particles
for idParticle in range(numParticle):
    x = [x0]
    y = [y0]
    theta = [np.random.rand()*2.0*np.pi]

    for iter in range(numStep - 1):
        tx = x[iter] + v0*np.cos(theta[iter])*dt
        ty = y[iter] + v0*np.sin(theta[iter])*dt 
        tt = theta[iter] + np.random.normal(0.0, sigma) 

        x.append(tx)
        y.append(ty)
        theta.append(tt)

    ax.plot(x, y)
    xs.append(x) 
    ys.append(y) 
    thetas.append(theta) 

ax.plot(x0, y0, "ro")
ax.set_aspect("equal")
plt.show()

In [ ]:
# compute auto-correlation
count = np.zeros(numStep)
r2 = np.zeros(numStep)

for idParticle in range(numParticle):
    for i in range(numStep):
        for j in range(i, numStep):
            dx = xs[idParticle][j] - xs[idParticle][i]
            dy = ys[idParticle][j] - ys[idParticle][i]

            r2[j - i] += dx**2.0 + dy**2.0 
            count[j - i] += 1

def func(x, a, b):
    return a*x**b

msd = r2/count

t = np.arange(dt, dt*numStep, dt)

fig, ax = plt.subplots()
ax.plot(t, msd[1:])
ax.plot(t, t, color="black", linestyle="dashdot", label="O(t)")
ax.plot(t, t**2.0, color="black", linestyle="dashed", label="O(t^2)")
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel("time")
ax.set_ylabel("MSD")
ax.legend()
plt.show()

In [ ]:
# compute auto-correlation
count = np.zeros(numStep)
correlation = np.zeros(numStep)

for idParticle in range(numParticle):
    for i in range(numStep):
        for j in range(i, numStep):
            ti = thetas[idParticle][i]
            tj = thetas[idParticle][j]

            correlation[j - i] += np.cos(ti)*np.cos(tj) + np.sin(ti)*np.sin(tj)
            count[j - i] += 1

def func(x, a):
    return np.exp(-a*x)

t = np.arange(0.0, dt*numStep, dt)
param, _ = curve_fit(func, t, correlation/count)
print("fitted parameter: ", param[0])

fig, ax = plt.subplots()
ax.plot(t, correlation/count)
ax.plot(t, func(t, param))
plt.show()